모듈 import

In [6]:
import tkinter as tk
from tkinter import ttk, messagebox
import webbrowser

상세 화면 클래스 정의

In [7]:
class RestaurantDetail(ttk.Frame):
    def __init__(self, parent, data: dict):
        super().__init__(parent, padding=12)
        self.data = data
        self._build()

    def _build(self):
        # ===== 상단: 이미지 + 가게 이름 =====
        header = ttk.Frame(self)
        header.pack(fill="x", pady=(0, 10))

        # 왼쪽: 이미지 플레이스홀더
        img_box = tk.Canvas(header, width=120, height=90, bd=0, highlightthickness=1, relief="ridge")
        img_box.pack(side="left", padx=(0, 10))
        img_box.create_text(60, 45, text="IMAGE", font=("맑은 고딕", 10))

        # 오른쪽: 이름 + 태그(선택)
        info_box = ttk.Frame(header)
        info_box.pack(side="left", fill="x", expand=True)

        name = self.data.get("name", "가게 이름")
        ttk.Label(info_box, text=name, font=("맑은 고딕", 14, "bold")).pack(anchor="w")

        tags = self.data.get("tags", [])
        if tags:
            ttk.Label(info_box, text=" · ".join(tags), foreground="#666").pack(anchor="w", pady=(2, 0))

        rating = self.data.get("rating")
        if isinstance(rating, (int, float)):
            ttk.Label(info_box, text=f"⭐ {rating:.1f}", foreground="#f39c12").pack(anchor="w", pady=(2, 0))

        # ===== 버튼 영역 (지도 / SNS) =====
        btn_row = ttk.Frame(self)
        btn_row.pack(fill="x", pady=(0, 10))

        def open_url(url, msg):
            if url:
                webbrowser.open(url)
            else:
                messagebox.showinfo("알림", f"{msg} 링크가 없습니다.")

        ttk.Button(
            btn_row,
            text="지도 열기",
            command=lambda: open_url(self.data.get("map_url"), "지도")
        ).pack(side="left", padx=(0, 5))

        ttk.Button(
            btn_row,
            text="SNS / 카페",
            command=lambda: open_url(self.data.get("sns_url"), "SNS/카페")
        ).pack(side="left", padx=5)

        # ===== 기본 정보 =====
        info_section = ttk.LabelFrame(self, text="기본 정보")
        info_section.pack(fill="x", pady=(0, 10))

        def info_row(label, value):
            row = ttk.Frame(info_section)
            row.pack(fill="x", pady=2)
            ttk.Label(row, text=label, width=8, anchor="w").pack(side="left")
            ttk.Label(row, text=value or "-", anchor="w").pack(side="left")

        info_row("전화", self.data.get("phone"))
        info_row("주소", self.data.get("address"))
        info_row("주차", "가능" if self.data.get("parking") else "불가")
        info_row("영업", self.data.get("hours_str"))

        # ===== 메뉴 =====
        menu_section = ttk.LabelFrame(self, text="메뉴")
        menu_section.pack(fill="x", pady=(0, 10))

        menu_list = self.data.get("menu", [])
        if not menu_list:
            ttk.Label(menu_section, text="등록된 메뉴가 없습니다.", foreground="#666").pack(anchor="w", padx=4, pady=4)
        else:
            lines = []
            for m in menu_list:
                name = m.get("name", "")
                price = m.get("price", "")
                desc = m.get("desc", "")
                line = f"- {name} ({price})"
                if desc:
                    line += f" : {desc}"
                lines.append(line)
            text = "\n".join(lines)
            ttk.Label(menu_section, text=text, justify="left").pack(anchor="w", padx=4, pady=4)

테스트용

In [8]:
def open_detail(data: dict):
    win = tk.Toplevel()
    win.title(data.get("name", "상세 보기"))
    win.geometry("600x500")
    RestaurantDetail(win, data).pack(fill="both", expand=True)
    win.transient()
    win.grab_set()
    win.focus_set()

In [9]:
demo = {
    "id": "bonjeon-0001",
    "name": "본전돼지국밥",
    "tags": ["돼지국밥 전문점", "부산역 인근", "대중적"],
    "rating": 4.1,
    "review_count": 4607,
    "price_range": "₩1~10,000",

    # 사진 경로는 현재 없음
    "photo_path": "",

    # 외부 링크
    "map_url": "https://maps.google.com/?q=부산+본전돼지국밥",
    "sns_url": "",

    # 기본 정보
    "phone": "051-441-2946",
    "address": "부산광역시 동구 중앙대로214번길 3-8",
    "parking": False,  # 구글 정보 기준 명확히 없음 → 일단 불가 처리
    "hours_str": "목 08:30 종료 · 금 09:00 영업 시작",

    # 메뉴 일단 간단히 대충 작성
    "menu": [
        {"name": "돼지국밥", "price": "7,000원", "desc": "기본 메뉴 (비공식)"},
        {"name": "특돼지국밥", "price": "9,000원", "desc": "고기 추가 (비공식)"},
        {"name": "수육", "price": "25,000원", "desc": "소/중/대 (비공식)"},
    ],

    # 리뷰 키워드는 지금 없으므로 빈 리스트로
    "keywords": [],
}


In [10]:
root = tk.Tk()
root.title("부산 국밥집 추천기 - 메인(데모)")
ttk.Button(root, text="정보 상세 보기", command=lambda: open_detail(demo)).pack(
    padx=20, pady=20
)
root.mainloop()
